In [15]:
from algorithm import chromatic_balls, deep_cluster, greedy_expansion, pivot, greedy_vote
from random import shuffle, random, choices, randrange, seed
from graph import Graph
from algorithm.util import most_frequent_color, shuffled
import queue as Q
from typing import List, Dict
import gurobipy as gp
from gurobipy import GRB
from dataset import read_dataset
import networkx as nx
import itertools as it
from scipy.sparse import coo_matrix

In [ ]:
# solve the LP relaxation
def solve_lp(graph: Graph): 

    model = gp.Model("CCC")
    
    colors = list(graph.colors())
    
    print("Adding variables...")
    v = model.addVars(graph.nodes(), colors, lb=0, ub=1, name="v")
    e = model.addVars(graph.node_pairs(), colors, lb=0, ub=1, name="e")
    
    print("Variables added.")
    L = len(colors)
     
    # objective function
    print("Adding objective function...")
    model.setObjective(gp.quicksum((e[a,b, graph.color_of(a,b)] if graph.has_edge(a,b)
                                   else gp.quicksum(1 - e[a,b, l] for l in colors))
                                   for a, b in graph.node_pairs()), GRB.MINIMIZE)
    
    print("Objective function added.")
    
    # boundary constraints
    print("Adding chromatic constraints...")
    model.addConstrs((gp.quicksum(v[a,l] for l in colors) == L-1
                      for a in graph.nodes()),
                     "chromatic")
    
    print("Adding edge-node constraints...")
    model.addConstrs((e[a,b,l] >= v[a, l]
                      for l in colors
                      for a, b in graph.node_pairs()
                      ),
                     "ev_1")
    model.addConstrs((e[a,b,l] >= v[b, l]
                      for l in colors
                      for a, b in graph.node_pairs()
                      ),
                     "ev_2")
    
    print("Preparing triangle constraints...")
    nodes = list(graph.nodes())
    idx_of = {key:i for i, key in enumerate(e.keys())}
    row, col, val = [], [], []
    r = 0
    for (a, b ,c) in it.combinations(nodes, 3):
      for l in colors:
        row += [r,r,r,r+1,r+1,r+1,r+2,r+2,r+2]
        col += [idx_of[(a, b, l)], idx_of[(b, c, l)], idx_of[(a, c, l)], 
                 idx_of[(b, c, l)], idx_of[(a, c, l)], idx_of[(a, b, l)],
                 idx_of[(a, b, l)], idx_of[(a, c, l)], idx_of[(b, c, l)]]
        val += [1, 1, -1] * 3
        r += 3
        
    A = coo_matrix((val, (row, col)), shape=(r, len(e.keys())))
    e_list = list(e.values())
    
    print("Adding triangle constraints...")
    model.addMConstr(A=A, x=e_list, sense=GRB.GREATER_EQUAL, b=[0]*r, name="triangle")
    
    print("Constraints added.")
    # solve the problem
    
    model.Params.Method = 2
    # model.Params.TimeLimit = 60
    
    model.optimize()
    print(v)
    
    status = model.status
    
    return model, status, v, e
  
# Rounding functions
def f(x,s):
  if s==2:      # 0
    if x<0.40:
      return 0
    elif x<0.495:
      return 0.78
    else:
      return min(1,0.348*(x-0.5)+0.85)
  elif s==1:    # +
    a=0.19
    b=0.5095
    if x<a:
      return 0
    elif x>b:
      return 1
    else:
      return ((x-a)/(b-a))**2
  else:         # -
      return x
  
def pivot_lp(graph: Graph, model: gp.Model, status: int, v: dict, e: dict): # 2.15-Approximation
    # model, status, v, e = solve_lp(graph)
    if status != GRB.OPTIMAL:
        print("No optimal solution found")
        return []
    print("Optimal solution found")
    print("Objective value:", model.ObjVal)
    
    colors = list(graph.colors())
    L = len(colors)
    partition = dict((color, []) for color in colors)

    clustering = []
    is_clustered = dict((i, False) for i in graph.nodes())

    for a in graph.nodes():
        has_color = False
        for color in colors:
            if v[a, color].X < 0.5:
                has_color = True
                partition[color].append(a)
                break
        if not has_color:
            is_clustered[a] = True
            clustering.append(([a], choices(colors, k=1)[0]))

    for color in colors:
        for center in shuffled(partition[color]):
            if is_clustered[center]: continue
            cluster = [center]
            is_clustered[center] = True
            for a in partition[color]:
                if is_clustered[a]: continue
                u1 = center
                u2 = a
                if u1 > u2:
                    u1, u2 = u2, u1
                prob = e[u1, u2, color].X
                sign = None
                if not graph.has_edge(u1, u2):
                    sign = 0
                elif graph.color_of(u1, u2) == color:
                    sign = 1
                else:
                    sign = 2
                prob = f(prob, sign)
                if random() < 1-prob:
                    is_clustered[a] = True
                    cluster.append(a)
            clustering.append((cluster, color))
    return clustering

In [88]:
# Graph from real life dataset
dataset = read_dataset('microsoft_academic')
graph = None
for graph_number, g in enumerate(dataset):
    if graph_number == 0:
        graph = g.subgraph(range(0, 300))
        break

# Graph from random dataset
# n = 100
# p = 0.9
# L = 5

# graph = Graph(nx.fast_gnp_random_graph(n, p))
# colors = list(range(L))

# for a,b in graph.edges():
#     edge_color = choices(colors, k=1)[0]
#     graph[a][b]['color'] = edge_color

# Solve the LP relaxation
model, status, v, e = solve_lp(graph)


Adding variables...
Variables added.
Adding objective function...
Objective function added.
Adding chromatic constraints...
Adding edge-node constraints...
Preparing triangle constraints...
Adding triangle constraints...


: 

In [87]:
seed(3)
print("pivot_lp: ",graph.error_of(pivot_lp(graph, model, status, v, e)))
print("chromatic_balls: ",graph.error_of(chromatic_balls.chromatic_balls(graph)))
print("deep_cluster: ",graph.error_of(deep_cluster.deep_cluster(graph)))
print("greedy_expansion: ",graph.error_of(greedy_expansion.greedy_expansion(graph)))
print("pivot: ",graph.error_of(pivot.pivot(graph)))
print("greedy_vote: ",graph.error_of(greedy_vote.greedy_vote(graph)))



Optimal solution found
Objective value: 33.0
pivot_lp:  35
chromatic_balls:  45
deep_cluster:  56
greedy_expansion:  35
pivot:  44
greedy_vote:  36
